<a href="https://colab.research.google.com/github/Prachii26/DeepLearningCMPE258/blob/main/Neural%20Network%20using%20numpy%2C%20keras%20and%20pytorch/Colabs/Colab_(C)_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Import libraries and set up device

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

Using device: cpu
PyTorch version: 2.9.0+cpu


In [ ]:
# Cell 2: Generate synthetic dataset and train/test split

# Generate synthetic data: y = sin(x1) * x2^2 + cos(x2) * x1 + 0.5 * x3
n_samples = 1000

x1 = np.random.uniform(-2, 2, n_samples)
x2 = np.random.uniform(-2, 2, n_samples)
x3 = np.random.uniform(-2, 2, n_samples)

X = np.column_stack([x1, x2, x3])
y = (np.sin(x1) * x2**2 + np.cos(x2) * x1 + 0.5 * x3).reshape(-1, 1)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Train/test split (80/20)
split_idx = int(0.8 * n_samples)
X_train, X_test = X_tensor[:split_idx], X_tensor[split_idx:]
y_train, y_test = y_tensor[:split_idx], y_tensor[split_idx:]

print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")

Training set: torch.Size([800, 3]), torch.Size([800, 1])
Test set: torch.Size([200, 3]), torch.Size([200, 1])


In [ ]:
# Cell 3: Create DataLoader for batched training

batch_size = 32

# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Train batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")
print(f"Batch size: {batch_size}")


Train batches: 25
Test batches: 7
Batch size: 32


In [ ]:
# Cell 4: Define the 3-layer neural network class

class ThreeLayerNet(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
        super(ThreeLayerNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.fc3 = nn.Linear(hidden2_size, output_size)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

print("3-layer neural network class defined successfully!")

3-layer neural network class defined successfully!


In [ ]:
# Cell 5: Instantiate model, loss function, and optimizer

# Network architecture
input_size = 3
hidden1_size = 10
hidden2_size = 8
output_size = 1

# Create model and move to device
model = ThreeLayerNet(input_size, hidden1_size, hidden2_size, output_size).to(device)

# Loss function
criterion = nn.MSELoss()

# Optimizer
learning_rate = 0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(f"Model: {model}")
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Loss function: MSE")
print(f"Optimizer: Adam with lr={learning_rate}")


Model: ThreeLayerNet(
  (fc1): Linear(in_features=3, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
  (tanh): Tanh()
)

Total parameters: 137
Loss function: MSE
Optimizer: Adam with lr=0.01


In [ ]:
# Cell 6: Training loop with progress printing

epochs = 100
print_every = 20

train_losses = []

print("Starting training...")
print("-" * 50)

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Forward pass
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Average loss for the epoch
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)

    if (epoch + 1) % print_every == 0:
        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {avg_loss:.6f}")

print("-" * 50)
print("Training completed!")

Starting training...
--------------------------------------------------
Epoch [20/100] - Loss: 0.046015
Epoch [40/100] - Loss: 0.022165
Epoch [60/100] - Loss: 0.015185
Epoch [80/100] - Loss: 0.011977
Epoch [100/100] - Loss: 0.009108
--------------------------------------------------
Training completed!


In [ ]:
# Cell 7: Evaluate on test set and print summary

model.eval()
test_loss = 0.0
all_predictions = []
all_targets = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)
        test_loss += loss.item()

        all_predictions.append(predictions.cpu())
        all_targets.append(batch_y.cpu())

# Calculate average test loss
avg_test_loss = test_loss / len(test_loader)

# Concatenate all predictions and targets
all_predictions = torch.cat(all_predictions)
all_targets = torch.cat(all_targets)

# Calculate R² score
ss_res = torch.sum((all_targets - all_predictions) ** 2)
ss_tot = torch.sum((all_targets - torch.mean(all_targets)) ** 2)
r2_score = 1 - (ss_res / ss_tot)

print("=" * 50)
print("EVALUATION SUMMARY")
print("=" * 50)
print(f"Final Training Loss: {train_losses[-1]:.6f}")
print(f"Test Loss: {avg_test_loss:.6f}")
print(f"Test R² Score: {r2_score.item():.4f}")
print(f"Test MAE: {torch.mean(torch.abs(all_targets - all_predictions)).item():.4f}")
print("=" * 50)

EVALUATION SUMMARY
Final Training Loss: 0.009108
Test Loss: 0.011476
Test R² Score: 0.9963
Test MAE: 0.0778


In [ ]:
# Cell 9: Final summary and completion report

print("=" * 70)
print(" " * 10 + "PYTORCH CLASS-BASED IMPLEMENTATION COMPLETE")
print("=" * 70)

print("\n✅ ASSIGNMENT REQUIREMENTS - ALL COMPLETED:")
print("-" * 70)
print("✓ Used torch.nn.Module for model definition")
print("✓ Used torch.nn.Linear for layers")
print("✓ Used torch.optim.Adam for optimizer")
print("✓ Used torch.nn.MSELoss for loss function")
print("✓ Used TensorDataset and DataLoader for batching")
print("✓ Used autograd (loss.backward() and optimizer.step())")
print("✓ 3-layer architecture: 3 inputs → 2 hidden → 1 output")
print("✓ Non-linear regression with MSE loss")

print("\n🧠 MODEL ARCHITECTURE:")
print("-" * 70)
print(f"Input Layer:      {input_size} features")
print(f"Hidden Layer 1:   {hidden1_size} neurons (ReLU)")
print(f"Hidden Layer 2:   {hidden2_size} neurons (Tanh)")
print(f"Output Layer:     {output_size} neuron (Linear)")
print(f"Total Parameters: {sum(p.numel() for p in model.parameters())}")

print("\n📊 TRAINING CONFIGURATION:")
print("-" * 70)
print(f"Dataset Size:     {n_samples} samples (train: {len(X_train)}, test: {len(X_test)})")
print(f"Batch Size:       {batch_size}")
print(f"Epochs:           {epochs}")
print(f"Learning Rate:    {learning_rate}")
print(f"Optimizer:        Adam")

print("\n🎯 FINAL RESULTS:")
print("-" * 70)
print(f"Training Loss:    {train_losses[-1]:.6f}")
print(f"Test Loss:        {avg_test_loss:.6f}")
print(f"Test R² Score:    {r2_score.item():.4f}")
print(f"Test MAE:         {torch.mean(torch.abs(all_targets - all_predictions)).item():.4f}")

print("\n" + "=" * 70)
print(" " * 20 + "🎉 ALL TASKS COMPLETED!")
print("=" * 70)

          PYTORCH CLASS-BASED IMPLEMENTATION COMPLETE

✅ ASSIGNMENT REQUIREMENTS - ALL COMPLETED:
----------------------------------------------------------------------
✓ Used torch.nn.Module for model definition
✓ Used torch.nn.Linear for layers
✓ Used torch.optim.Adam for optimizer
✓ Used torch.nn.MSELoss for loss function
✓ Used TensorDataset and DataLoader for batching
✓ Used autograd (loss.backward() and optimizer.step())
✓ 3-layer architecture: 3 inputs → 2 hidden → 1 output
✓ Non-linear regression with MSE loss

🧠 MODEL ARCHITECTURE:
----------------------------------------------------------------------
Input Layer:      3 features
Hidden Layer 1:   10 neurons (ReLU)
Hidden Layer 2:   8 neurons (Tanh)
Output Layer:     1 neuron (Linear)
Total Parameters: 137

📊 TRAINING CONFIGURATION:
----------------------------------------------------------------------
Dataset Size:     1000 samples (train: 800, test: 200)
Batch Size:       32
Epochs:           100
Learning Rate:    0.01
Optim